In [0]:
# Databricks notebook source
# ==========================================================
# EXECUTE JOB DDL - Proyecto Liga 1 Perú
# Recibe nombre de archivo DDL y lo ejecuta
# ==========================================================

from env_setup import *
from utils_liga1 import get_dbutils
import os
import re


spark = SparkSession.builder.getOrCreate()
dbutils = get_dbutils()

dbutils.widgets.text("prm_name_ddl", "")
prm_name_ddl= dbutils.widgets.get("prm_name_ddl") 

print(f"Archivo DDL recibido: {prm_name_ddl}")

container_name = dbutils.secrets.get(scope="secretliga1", key="filesystemname")
adls_account_name = dbutils.secrets.get(scope="secretliga1", key="storageaccount")
catalog_name = "adbliga1futbol" 


In [0]:
def execute_ddl_file(file_path):
    """Ejecuta un archivo DDL reemplazando las variables"""
    try:
        with open(file_path, 'r') as file:
            ddl_content = file.read()
        
        # Reemplazar variables
        ddl_content = ddl_content.replace("${catalog_name}", catalog_name)
        ddl_content = ddl_content.replace("${container_name}", container_name)
        ddl_content = ddl_content.replace("${storage_account}", adls_account_name)
        
        print(f"Ejecutando DDL desde: {file_path}")
        print("=" * 50)
        print(ddl_content)
        print("=" * 50)
        
        spark.sql(ddl_content)
        print(f"DDL ejecutado exitosamente: {os.path.basename(file_path)}")
        return True
        
    except Exception as e:
        print(f"Error ejecutando DDL {file_path}: {str(e)}")
        raise

def process_ddl_files(ddl_name):
    """Procesa todos los archivos DDL para un objeto específico"""
    # USAR get_workspace_path PARA CONSTRUIR LA RUTA CORRECTA
    base_path = get_workspace_path(f"ddl_deploy/{ddl_name}")
    
    print(f"Buscando en ruta: {base_path}")
    
    if not os.path.exists(base_path):
        print(f"No se encuentra la ruta: {base_path}")
        print(f"Directorio actual: {os.getcwd()}")
        print(f"Contenido de ddl_deploy: {os.listdir(get_workspace_path('ddl_deploy')) if os.path.exists(get_workspace_path('ddl_deploy')) else 'No existe'}")
        return False
    
    # Buscar archivos SQL en el directorio
    sql_files = [f for f in os.listdir(base_path) if f.endswith('.sql')]
    print(f"Archivos SQL encontrados: {sql_files}")
    
    if not sql_files:
        print(f"No se encontraron archivos SQL en: {base_path}")
        return False
    
    # Identificar archivos basados en el nombre del parámetro
    table_file = f"{ddl_name}.sql"  # Ej: ddl_md_catalogo_equipos.sql
    view_file = f"{ddl_name}_view.sql"  # Ej: ddl_md_catalogo_equipos_view.sql
    
    print(f"Buscando tabla: {table_file}")
    print(f"Buscando vista: {view_file}")
    
    # Ejecutar primero la tabla si existe
    if table_file in sql_files:
        file_path = os.path.join(base_path, table_file)
        execute_ddl_file(file_path)
    else:
        print(f"No se encontró archivo de tabla: {table_file}")
    
    # Ejecutar luego la vista si existe
    if view_file in sql_files:
        file_path = os.path.join(base_path, view_file)
        execute_ddl_file(file_path)
    else:
        print(f"No se encontró archivo de vista: {view_file}")
    
    # Ejecutar cualquier otro archivo SQL que no sea tabla o vista principal
    other_files = [f for f in sql_files if f not in [table_file, view_file]]
    for sql_file in other_files:
        file_path = os.path.join(base_path, sql_file)
        print(f"Ejecutando archivo adicional: {sql_file}")
        execute_ddl_file(file_path)
    
    return True

In [0]:
# Ejecutar el proceso principal
try:
    print("Iniciando proceso DDL...")
    print(f"Directorio de trabajo actual: {os.getcwd()}")
    print(f"Ruta repo root: {detect_repo_root()}")
    
    # Verificar si existe el directorio base ddl_deploy
    ddl_deploy_base = get_workspace_path("ddl_deploy")
    print(f"Ruta base DDL: {ddl_deploy_base}")
    print(f"Contenido de ddl_deploy: {os.listdir(ddl_deploy_base) if os.path.exists(ddl_deploy_base) else 'NO EXISTE'}")
    
    success = process_ddl_files(prm_name_ddl)
    if success:
        print(f"Proceso DDL completado exitosamente para: {prm_name_ddl}")
    else:
        print(f"Falló el proceso DDL para: {prm_name_ddl}")
        
except Exception as e:
    print(f"Exception: Fallo en ejecución DDL: {str(e)}")
    raise